In [1]:
from anthropic import AnthropicBedrock
from application import Application
from services import CompilerService
import tempfile

In [2]:
compiler = CompilerService()
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [ ]:

tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [4]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [5]:
my_bot = application.create_bot(application_description)

Compiling TypeSpec...
Generating Typescript Schema Definitions...


AttributeError: 'CompilerService' object has no attribute 'compile_typescript'

In [19]:
my_bot

{'typespec': {'message': {'role': 'assistant',
   'content': '<reasoning>\nFor a gym tracking bot, I expect users to send messages like:\n"I did 3 sets of bench press with 80kg" -> recordExercise()\n"Show my progress for bench press" -> getProgress()\n"Suggest a 30-minute workout for chest with dumbbells" -> suggestRoutine()\n\nKey aspects to track:\n1. Exercise details (name, sets, reps, weight)\n2. Equipment used\n3. Time spent\n4. Body part targeted\n5. Progress over time\n6. Personalized routine suggestions based on:\n   - Available equipment\n   - Time constraints\n   - Target muscle groups\n   - Previous performance\n</reasoning>\n\n<typespec>\nmodel Exercise {\n    name: string;\n    sets: integer;\n    reps: integer;\n    weight: float;\n    equipment: string;\n    bodyPart: string;\n    duration: duration;\n}\n\nmodel WorkoutSession {\n    exercises: Exercise[];\n    totalDuration: duration;\n    timestamp: utcDateTime;\n}\n\nmodel RoutineRequest {\n    availableEquipment: str

In [20]:
application.generation_dir



'/tmp/tmpt8llr03k/generated/generation-1'

In [21]:
#tempdir.cleanup()